<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=4673ab412dc479769c450c7bbd8172db943fa5d0fe2c414609c57c85d90ee1ed
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-20 10:20:04
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.40 C
-------------------
Today PnL: -8.13 K (-0.06%)
Current PnL: -21.31 L (-14.2%)
CY Booked + Current PnL: -9.41 L (-6.27%)
-------------------
Total profit:  1.93 L
Total loss:  -23.24 L
-------------------
Total Booked + Current PnL: 17.57 L (14.27%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.49%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 88.47 L (63.12%)
Deployed:  1.23 C
Current:  1.40 C
CAGR/XIRR %: 7.66%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-18.75,62.0,X-LC,3.95,217186.0,12686.0,10599.0,0.49,6.20,4.88,11.39,10.0,1.20,1.56,15.71,X40N,ATH,PHARMA
56,RELIANCE,1533.00,-8.78,73.0,X-LC,2.88,228134.0,17768.0,11019.0,3.22,8.45,4.83,13.68,37.0,1.61,1.64,26.37,XY25,NTT,REFINERIES
76,TTKPRESTIG,770.00,94.88,46.0,M-SC,1.71,83997.0,-16780.0,16875.0,-0.16,-16.65,20.09,0.09,245.0,-0.99,0.60,10.35,OX40N,NTT,DURABLES
33,ICICIGI,2260.25,-12.68,67.0,X-MC,2.71,150368.0,14235.0,16886.0,1.55,10.46,11.23,22.86,91.0,0.84,1.08,24.39,X40,ATH,INSURANCE
83,VOLTAS,1530.00,0.74,55.0,X-MC,1.45,211410.0,19668.0,18097.0,-1.21,10.26,8.56,19.69,99.0,1.09,1.52,17.75,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RELIANCE,1533.00,-8.78,73.0,X-LC,2.88,228134.0,17768.0,11019.0,3.22,8.45,4.83,13.68,37.0,1.61,1.64,26.37,XY25,NTT,REFINERIES
45,KPIGREEN,731.05,4.03,63.0,H-SC,5.86,124034.0,-1263.0,60194.0,2.87,-1.01,48.53,47.03,141.0,-0.02,0.89,55.30,MH,ATH,POWER
11,BANKINDIA,190.00,-25.47,59.0,H-MC,12.41,193619.0,13811.0,98610.0,2.26,7.68,50.93,62.52,88.0,0.14,1.39,44.39,XR,NTT,BANKS
78,UNIONBANK,163.00,-11.63,55.0,M-LC,8.70,158634.0,17794.0,26698.0,2.07,12.63,16.83,31.59,66.0,0.67,1.14,42.47,XY24,NTT,BANKS
20,COFFEEDAY,80.00,-43.66,53.0,L-SC,29.68,82944.0,-30605.0,70660.0,1.96,-26.95,85.19,35.27,268.0,-0.43,0.60,103.01,XR,NTT,HOTELS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.00,-40.79,61.0,M-SC,4.20,100148.0,8650.0,70104.0,-3.55,9.45,70.00,86.07,223.0,0.12,0.72,45.43,XR,NTT,DURABLES
31,HINDZINC,730.22,29.16,52.0,M-LC,10.08,210758.0,5682.0,106896.0,-3.15,2.77,50.72,54.89,52.0,0.05,1.52,27.53,X5K,ATH,METALS
25,GREENPANEL,537.00,230.10,54.0,M-SC,4.17,147352.0,-35726.0,114699.0,-2.66,-19.51,77.84,43.14,230.0,-0.31,1.06,38.60,XY24,NTT,MISC
70,TANLA,1963.11,-24.85,40.0,H-SC,18.83,201042.0,-69719.0,399671.0,-2.60,-25.75,198.80,121.86,133.0,-0.17,1.45,57.27,AR,ATH,IT
57,REPCOHOME,880.00,-58.27,55.0,H-SC,2.49,238702.0,-47803.0,305133.0,-2.03,-16.68,127.83,89.82,134.0,-0.16,1.72,26.29,XY24,NTT,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,INDIAMART,4810.62,-55.95,37.0,H-SC,7.74,123225.0,-111.0,131740.0,-0.81,-0.09,106.91,106.72,119.0,-0.0,0.89,23.27,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,108.87,61.0,H-SC,12.69,131895.0,-10584.0,30098.0,0.27,-7.43,22.82,13.70,163.0,-0.35,0.95,55.67,OX40N,NTT,BANKS
44,KANSAINER,340.0,-67.42,55.0,H-SC,1.87,225189.0,-44478.0,80820.0,-1.20,-16.49,35.89,13.47,138.0,-0.55,1.62,14.50,XY24,NTT,PAINTS
76,TTKPRESTIG,770.0,94.88,46.0,M-SC,1.71,83997.0,-16780.0,16875.0,-0.16,-16.65,20.09,0.09,245.0,-0.99,0.60,10.35,OX40N,NTT,DURABLES
17,CERA,9475.0,-24.34,38.0,H-SC,2.49,137931.0,-37972.0,80000.0,-0.08,-21.59,58.00,23.89,149.0,-0.47,0.99,19.12,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-24.21,46.0,H-MC,7.51,102572.0,-29263.0,71493.0,0.54,-22.20,69.70,32.03,98.0,-0.41,0.74,14.89,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,47.87,53.0,M-LC,6.03,207598.0,4428.0,37700.0,0.67,2.18,18.16,20.74,55.0,0.12,1.49,7.84,XY25,NTT,FINANCE
31,HINDZINC,730.22,29.16,52.0,M-LC,10.08,210758.0,5682.0,106896.0,-3.15,2.77,50.72,54.89,52.0,0.05,1.52,27.53,X5K,ATH,METALS
78,UNIONBANK,163.00,-11.63,55.0,M-LC,8.70,158634.0,17794.0,26698.0,2.07,12.63,16.83,31.59,66.0,0.67,1.14,42.47,XY24,NTT,BANKS
11,BANKINDIA,190.00,-25.47,59.0,H-MC,12.41,193619.0,13811.0,98610.0,2.26,7.68,50.93,62.52,88.0,0.14,1.39,44.39,XR,NTT,BANKS
84,WHIRLPOOL,2270.00,-40.79,61.0,M-SC,4.20,100148.0,8650.0,70104.0,-3.55,9.45,70.00,86.07,223.0,0.12,0.72,45.43,XR,NTT,DURABLES


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BANKINDIA,190.00,-25.47,59.0,H-MC,12.41,193619.0,13811.0,98610.0,2.26,7.68,50.93,62.52,88.0,0.14,1.39,44.39,XR,NTT,BANKS
84,WHIRLPOOL,2270.00,-40.79,61.0,M-SC,4.20,100148.0,8650.0,70104.0,-3.55,9.45,70.00,86.07,223.0,0.12,0.72,45.43,XR,NTT,DURABLES
31,HINDZINC,730.22,29.16,52.0,M-LC,10.08,210758.0,5682.0,106896.0,-3.15,2.77,50.72,54.89,52.0,0.05,1.52,27.53,X5K,ATH,METALS
36,INDIAMART,4810.62,-55.95,37.0,H-SC,7.74,123225.0,-111.0,131740.0,-0.81,-0.09,106.91,106.72,119.0,-0.00,0.89,23.27,AR,ATH,MISC
85,WIPRO,420.00,-15.15,43.0,M-LC,6.01,149160.0,-1785.0,111244.0,-0.13,-1.18,74.58,72.51,53.0,-0.02,1.07,4.77,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-53.83,14.0,X-LC,9.58,145018.0,-131505.0,241571.0,0.73,-47.56,166.58,39.81,54.0,-0.54,1.04,2.21,XY24,NTT,AUTO
51,QUESS,986.00,-48.76,31.0,X-SC,38.82,50896.0,-14110.0,165040.0,-0.73,-21.71,324.27,232.18,198.0,-0.09,0.37,0.00,XY24,NTT,MISC
28,HAVELLS,2069.16,-7.45,39.0,X-MC,2.42,229507.0,-18224.0,93272.0,-1.36,-7.36,40.64,30.30,92.0,-0.20,1.65,4.96,X40,ATH,ELECTRICAL
12,BATAINDIA,2096.00,-36.15,42.0,X-SC,4.30,95516.0,-33154.0,78447.0,0.04,-25.77,82.13,35.20,219.0,-0.42,0.69,9.63,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-14.90,42.0,X-MC,5.45,179449.0,-21057.0,118203.0,-0.01,-10.50,65.87,48.45,90.0,-0.18,1.29,3.48,X40N,ATH,FINANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-2.61,46.0,H-LC,0.37,156430.0,-29665.0,77151.0,0.24,-15.94,49.32,25.51,15.0,-0.38,1.12,15.28,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,0.74,55.0,X-MC,1.45,211410.0,19668.0,18097.0,-1.21,10.26,8.56,19.69,99.0,1.09,1.52,17.75,XY25,NTT,AC
28,HAVELLS,2069.16,-7.45,39.0,X-MC,2.42,229507.0,-18224.0,93272.0,-1.36,-7.36,40.64,30.30,92.0,-0.20,1.65,4.96,X40,ATH,ELECTRICAL
41,ITC,452.00,-37.50,61.0,X-LC,2.60,202299.0,2161.0,18733.0,0.38,1.08,9.26,10.44,4.0,0.12,1.45,6.70,X40,NTT,FMCG
33,ICICIGI,2260.25,-12.68,67.0,X-MC,2.71,150368.0,14235.0,16886.0,1.55,10.46,11.23,22.86,91.0,0.84,1.08,24.39,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.76,31.0,X-SC,38.82,50896.0,-14110.0,165040.0,-0.73,-21.71,324.27,232.18,198.0,-0.09,0.37,0.00,XY24,NTT,MISC
72,TATAMOTORS,1065.00,-53.83,14.0,X-LC,9.58,145018.0,-131505.0,241571.0,0.73,-47.56,166.58,39.81,54.0,-0.54,1.04,2.21,XY24,NTT,AUTO
2,ACC,3906.00,-38.50,43.0,X-MC,3.23,182790.0,-55161.0,207814.0,-0.26,-23.18,113.69,64.15,174.0,-0.27,1.31,2.54,XY24,BTT,CEMENT
9,BAJAJHFL,181.50,-14.90,42.0,X-MC,5.45,179449.0,-21057.0,118203.0,-0.01,-10.50,65.87,48.45,90.0,-0.18,1.29,3.48,X40N,ATH,FINANCE
73,TCS,4389.97,-29.33,50.0,X-LC,12.75,282188.0,-63768.0,130456.0,1.34,-18.43,46.23,19.28,1.0,-0.49,2.03,4.32,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.76,31.0,X-SC,38.82,50896.0,-14110.0,165040.0,-0.73,-21.71,324.27,232.18,198.0,-0.09,0.37,0.00,XY24,NTT,MISC
55,RELAXO,1176.00,-39.88,42.0,X-SC,4.56,81921.0,-63239.0,139167.0,-0.57,-43.57,169.88,52.31,136.0,-0.45,0.59,9.66,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-36.15,42.0,X-SC,4.30,95516.0,-33154.0,78447.0,0.04,-25.77,82.13,35.20,219.0,-0.42,0.69,9.63,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-6.49,46.0,X-SC,0.85,106400.0,4396.0,39528.0,0.11,4.31,37.15,43.06,122.0,0.11,0.77,22.16,X40N,ATH,MISC
32,HONAUT,58357.33,-22.63,49.0,X-SC,2.55,109035.0,-18903.0,66032.0,0.10,-14.78,60.56,36.84,143.0,-0.29,0.78,11.63,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-29.33,50.0,X-LC,12.75,282188.0,-63768.0,130456.0,1.34,-18.43,46.23,19.28,1.0,-0.49,2.03,4.32,X40,ATH,IT
39,INFY,2275.00,-19.96,46.0,X-LC,7.62,311768.0,-1120.0,172813.0,1.57,-0.36,55.43,54.87,3.0,-0.01,2.24,6.27,X40,BTT,IT
41,ITC,452.00,-37.50,61.0,X-LC,2.60,202299.0,2161.0,18733.0,0.38,1.08,9.26,10.44,4.0,0.12,1.45,6.70,X40,NTT,FMCG
82,VBL,671.64,-19.29,49.0,X-LC,10.53,290896.0,-24946.0,137623.0,-1.21,-7.90,47.31,35.67,5.0,-0.18,2.09,4.86,X40N,ATH,FMCG
1,ABB,7934.00,-39.77,53.0,H-LC,6.89,251520.0,-10099.0,129306.0,0.73,-3.86,51.41,45.57,7.0,-0.08,1.81,7.02,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6866.67,61.0,L-SC,6.41,76431.0,-17379.0,93995.0,1.07,-18.53,122.98,81.67,269.0,-0.18,0.55,48.01,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-43.66,53.0,L-SC,29.68,82944.0,-30605.0,70660.0,1.96,-26.95,85.19,35.27,268.0,-0.43,0.60,103.01,XR,NTT,HOTELS
48,MASFIN,398.61,-20.51,43.0,H-SC,9.42,90150.0,-7830.0,29434.0,1.04,-7.99,32.65,22.05,152.0,-0.27,0.65,31.04,XR,ATH,FINANCE
84,WHIRLPOOL,2270.00,-40.79,61.0,M-SC,4.20,100148.0,8650.0,70104.0,-3.55,9.45,70.00,86.07,223.0,0.12,0.72,45.43,XR,NTT,DURABLES
45,KPIGREEN,731.05,4.03,63.0,H-SC,5.86,124034.0,-1263.0,60194.0,2.87,-1.01,48.53,47.03,141.0,-0.02,0.89,55.30,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.0,-187.50,71.0,M-SC,19.68,152028.0,2808.0,141371.0,1.30,1.88,92.99,96.62,208.0,0.02,1.09,64.53,XY25,NTT,FINANCE
4,ANGELONE,3033.0,15.82,70.0,X-SC,8.29,205328.0,14322.0,43386.0,0.30,7.50,21.13,30.21,157.0,0.33,1.48,29.94,X40N,NTT,FINANCE
67,STARHEALTH,761.0,21.26,66.0,H-SC,9.79,270014.0,-11109.0,143971.0,-1.51,-3.95,53.32,47.26,144.0,-0.08,1.94,44.67,XY24,NTT,INSURANCE
66,SONACOMS,1006.0,-35.10,64.0,M-SC,11.43,80850.0,-20309.0,95201.0,-0.04,-20.08,117.75,74.03,202.0,-0.21,0.58,14.01,AR,BTT,AUTO
77,UJJIVANSFB,60.0,108.87,61.0,H-SC,12.69,131895.0,-10584.0,30098.0,0.27,-7.43,22.82,13.70,163.0,-0.35,0.95,55.67,OX40N,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.70
1,25,44.28
2,50,75.87


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.18
LC    34.01
MC    21.81
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.49
X40      14.89
XY25     12.43
XR       10.94
X40N     10.25
AR        8.03
OX40N     7.61
X200      1.83
X5K       1.52
SR        1.12
MH        0.89
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.31
X-LC    23.00
X-MC    16.08
M-SC    12.70
M-LC     5.22
H-LC     4.76
X-SC     4.68
H-MC     3.83
M-MC     1.59
L-SC     1.49
L-LC     1.03
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.25,-3.55,37.68
IT,12.76,-18.54,82.52
FINANCE,12.19,-11.50,62.04
MISC,6.85,-20.89,87.87
BANKS,6.38,-12.43,71.57
PAINTS,5.71,-16.27,33.60
ELECTRICAL,5.36,-10.30,49.00
INSURANCE,3.99,0.30,36.97
AUTO,2.81,-46.67,116.79


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3332293.0,22
XR,1388256.0,14
AR,1213160.0,9
X40,766455.0,10
X40N,602799.0,8
OX40N,587927.0,10
XY25,452377.0,8
SR,266170.0,2
X5K,106896.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3481750.0,25
M-SC,1273015.0,15
X-MC,1090342.0,11
X-LC,1037869.0,13
X-SC,531600.0,6
H-MC,346858.0,3
M-LC,282538.0,4
H-LC,277449.0,3
L-SC,255190.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1202359.0      6
           AR         843901.0      5
           XR         834329.0      7
M-SC       XY24       697621.0      6
X-MC       XY24       588395.0      4
X-LC       X40        471807.0      5
           XY24       304267.0      2
X-SC       X40N       288113.0      4
H-SC       OX40N      274797.0      4
           SR         266170.0      2
X-MC       X40        216201.0      4
H-LC       AR         206457.0      2
X-LC       X40N       196483.0      3
H-MC       XY24       176755.0      1
M-MC       XY24       171158.0      1
X-MC       XY25       167543.0      2
X-SC       XY24       165040.0      1
L-SC       XR         164655.0      2
M-SC       AR         162802.0      2
           OX40N      151102.0      4
           XY25       141371.0      1
           XR         120119.0      2
X-MC       X40N       118203.0      1
M-LC       XR         111244.0      1
           X5K        106896.0      1
H-MC       XR          98610.0      1
L-SC       OX40N       90535.0      1
X-SC       X40         78447.0      1
H-MC       OX40N       71493.0      1
H-LC       X200        70992.0      1
X-LC       XY25        65312.0      3
H-SC       MH          60194.0      1
L-MC       XR          59299.0      1
L-LC       XY25        40451.0      1
M-LC       XY25        37700.0      1
           XY24        26698.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
